In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

random_state = 67

bank_marketing = fetch_ucirepo(id=222)
X = bank_marketing.data.features
y = bank_marketing.data.targets

# EDA

### Target

In [ ]:
print(type(y))
print(y.shape)
print(y.dtypes)

In [ ]:
y_series = y.iloc[:,0]
print(y_series.value_counts())
print(y_series.value_counts(normalize=True))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(data = y, x = y_series, palette='hls')
plt.title("Distribution of y")
plt.show()
plt.close()

Target jest mocno niezbalansowany, przez co podczas liczenia metryk nie będziemy opierać się na accuracy(baseline ~0.88 dla 'no'), będziemy raczej chcieli patrzeć na recall/f1 dla 'yes'. Threshold 0.5 też nie będzie dobry przy takim rozkładzie y.

## Features

In [ ]:
X_eda = X.copy()
print(X_eda.head())
print(X_eda.dtypes)
print(X_eda.shape)

### Missing values

In [ ]:
print(X_eda.isna().sum())

In [ ]:
missing_cols = X_eda.isna().sum().to_frame()
missing_cols = missing_cols[missing_cols.loc[:,0] > 0].index
for col in missing_cols:
    print('unknown' in X_eda.loc[:,col].values)

Kolumny gdzie są brakujące dane maja dtype = object oraz nie mają w sobie defaultowo 'unknown', także zamienimy brakujące wartośći na 'unknown'

In [ ]:
X_eda.loc[:,missing_cols] = X_eda.loc[:,missing_cols].fillna('unknown')
print(X_eda.isna().sum())

### Outliers

In [ ]:
num_cols = X_eda.select_dtypes('number').columns.tolist()
cat_cols = X_eda.select_dtypes('object').columns.tolist()
print(X_eda.loc[:,num_cols].describe())

Duration mówi nam o czasie rozmowy, a znamy ją dopiero po zakończeniu jej, dlatego usuwamy ją.

In [ ]:
df_with_y = X_eda.copy()
df_with_y['y'] = y_series
if 'duration' in df_with_y.columns:
    df_with_y.drop(columns=['duration'], inplace=True)

num_cols_without_duration = df_with_y.select_dtypes('number').columns.tolist()

for col in num_cols_without_duration:
    sns.violinplot(data = df_with_y, x="y", y=col, cut = 0, inner='quartile')
    plt.title(f"Distribution of {col}")
    plt.show()
    plt.close()

Decyzje
- Age - nie robimy nic
- Balance - nie robimy nic
- Day of the week - nie robimy nic
- Campaing - nie robimy nic
- Pdays - ma specjalną flagę -1 jeżeli nie był kontaktowany także możemy dodać osoby feature binarny który będzie nam mówił czy kontaktowaliśmy się
- Previous - nie robimy nic 

Balance, campaing, previous i age mają duże zakresy także w pipelineach do modeli liniowych będziemy mogli je popoprawiać.

In [ ]:
df_pdays = df_with_y[df_with_y['y'] != -1].copy()
sns.violinplot(data=df_pdays, x='y', y='pdays', cut=0, inner='quartile')
plt.title('Pdays without -1')
plt.show()
plt.close()

### Korelacje

In [ ]:
from sklearn.feature_selection import mutual_info_classif

y_bin = (y_series == "yes").astype('int')

mi_num = mutual_info_classif(X_eda.loc[:,num_cols_without_duration], y=y_bin, random_state=random_state)
mi_num = pd.Series(mi_num, index=num_cols_without_duration).sort_values(ascending=False)
print(mi_num.head())

In [ ]:
from scipy.stats import chi2_contingency

rows = []
for col in cat_cols:
    ct = pd.crosstab(X_eda.loc[:,col], y_series)
    chi2, p, dof, freq = chi2_contingency(ct) 
    rows.append({
        'feature': col,
        'chi2': chi2,
        'p_value': p,
        'dof': dof,
        'n_categories': ct.shape[0]
    })

chi2_rank = pd.DataFrame(rows).sort_values(by='chi2', ascending=False)
print(chi2_rank.head(10))

In [ ]:
top_cat_col = chi2_rank.head(5)['feature'].tolist()

df_with_y['y_bin'] = y_bin

def yes_rate_by_cat(df, col, min_n = 50):
    stats = (
        df.groupby(col)["y_bin"]
        .agg(n='size', yes_rate = "mean")
        .query('n>= @min_n')
        .sort_values('yes_rate', ascending=False)
    )

    return stats

for col in top_cat_col:
    stats = yes_rate_by_cat(df_with_y, col)
    sns.barplot(data=stats, x=col, y='yes_rate')
    plt.xticks(rotation = 45)
    plt.show()
    plt.close()